In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT_v, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()
print("-"*70)

# models
only = gp.Model("only")
only.Params.OutputFlag = 0
only.setParam("MIPGap", 1e-7)

x = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
zeta = only.addVars(I, T, S, vtype=GRB.BINARY, name="z")
only.update()

obj = gp.quicksum(P_DA[t] * x[i, t] for i in range(I) for t in range(T)) + gp.quicksum(
    1/S * (P_RT_v[t, s] * yp[i, t, s] - P_PN[t] * ym[i, t, s]) for i in range(I) for t in range(T) for s in range(S))
only.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    only.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s])
    only.addConstr(yp[i, t, s] <= R[i, t, s])
    only.addConstr(yp[i, t, s] <= M1 * zeta[i, t, s])
    only.addConstr(ym[i, t, s] <= M1 * (1 - zeta[i, t, s]))
only.optimize()

if only.status == GRB.OPTIMAL:
    print(f"[ONLY] {only.objVal}")
    total_da_profit_obj = 0
    for i, t in product(range(I), range(T)):
        total_da_profit_obj += P_DA[t] * x[i,t].x
    total_rt_profit_obj = 0
    for i, t, s in product(range(I), range(T), range(S)):
       rt_profit_obj = P_RT_v[t, s] * yp[i, t, s].x
       total_rt_profit_obj += 1/S * rt_profit_obj
    total_penalty_cost_obj = 0
    for i, t, s in product(range(I), range(T), range(S)):
        penalty_cost_obj = P_PN[t] * ym[i, t, s].x
        total_penalty_cost_obj += 1/S * penalty_cost_obj
    total_system_profit_obj = total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj
    print(f"DA: {total_da_profit_obj:.2f}")
    print(f"RT: {total_rt_profit_obj:.2f}")
    print(f"Penalty: {total_penalty_cost_obj:.2f}")
    print("-"*70)
else:
    print("No optimal solution found.")
    
# ----------------------------------

agg = gp.Model("agg")
agg.Params.OutputFlag = 0
agg.setParam("MIPGap", 1e-7)

a = agg.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
bp = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
bm = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
mu = agg.addVars(T, S, vtype=GRB.BINARY, name="z")
agg.update()

obj = gp.quicksum(P_DA[t] * a[t] for t in range(T)) + gp.quicksum(
    1/S * (P_RT_v[t, s] * bp[t, s] - P_PN[t] * bm[t, s]) for t in range(T) for s in range(S))
agg.setObjective(obj, GRB.MAXIMIZE)

for t, s in product(range(T), range(S)):
    agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - a[t] == bp[t, s] - bm[t, s])
    agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) >= bp[t, s])
    agg.addConstr(bp[t, s] <= M2 * mu[t, s])
    agg.addConstr(bm[t, s] <= M2 * (1 - mu[t, s]))
agg.optimize()

if agg.status == GRB.OPTIMAL:
    print(f"[AGG] {agg.objVal}")
    total_da_profit_obj = 0
    for t in range(T):
        total_da_profit_obj += P_DA[t] * a[t].x  
    total_rt_profit_obj = 0
    for t in range(T):
        for s in range(S):
            rt_profit_obj = P_RT_v[t, s] * bp[t, s].x  
            total_rt_profit_obj += 1/S * rt_profit_obj
    total_penalty_cost_obj = 0
    for t in range(T):
        for s in range(S):
            penalty_cost_obj = P_PN[t] * bm[t, s].x
            total_penalty_cost_obj += 1/S * penalty_cost_obj
    total_system_profit_obj = total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj
    print(f"DA: {total_da_profit_obj:.2f}")
    print(f"RT: {total_rt_profit_obj:.2f}")
    print(f"Penalty: {total_penalty_cost_obj:.2f}")
    print("-"*70)
else:
    print("No optimal solution found.")
    
x_v   = np.array([[only.getVarByName(f"x[{i},{t}]").X for t in range(T)] for i in range(I)])
yp_v = np.array([[[only.getVarByName(f"y_plus[{i},{t},{s}]").X for s in range(S)] for t in range(T)] for i in range(I)])
ym_v = np.array([[[only.getVarByName(f"y_minus[{i},{t},{s}]").X for s in range(S)] for t in range(T)] for i in range(I)])
a_v  = np.array([agg.getVarByName(f"alpha[{t}]").X for t in range(T)])
bp_v = np.array([[agg.getVarByName(f"beta_plus[{t},{s}]").X for s in range(S)] for t in range(T)])
bm_v = np.array([[agg.getVarByName(f"beta_minus[{t},{s}]").X for s in range(S)] for t in range(T)])

✅ 총 3개 파일을 불러왔습니다: 1201.csv, 401.csv, 89.csv
📊 데이터 Shape: I=3, T=24, S=30
✅ 시뮬레이션 초기화 완료: S=30, Randomness='high', M1=773.00, M2=1199.00
----------------------------------------------------------------------
[ONLY] 755303.6144954375
DA: 167420.54
RT: 613507.35
Penalty: 25624.27
----------------------------------------------------------------------
[AGG] 763721.4639892324
DA: 193532.27
RT: 588297.17
Penalty: 18107.98
----------------------------------------------------------------------


In [27]:
idx = 10
xv   = x_v
ypv  = yp_v[:, :, idx]
ymv  = ym_v[:, :, idx]
av   = a_v
bpv  = bp_v[:, idx]
bmv  = bm_v[:, idx]
P_RT = P_RT_v[:, idx]

In [28]:
delta_DA = np.zeros(T)
delta_BMp = np.zeros(T)
delta_BMm = np.zeros(T)
lambd = np.zeros((I, T))
total_surplus = np.zeros(T)

for t in range(T):
    delta_DA[t]  = av[t]  - np.sum(xv[:, t])
    delta_BMp[t] = bpv[t] - np.sum(ypv[:, t])
    delta_BMm[t] = bmv[t] - np.sum(ymv[:, t])
    total_surplus[t]   = P_DA[t] * delta_DA[t] + P_RT[t] * delta_BMp[t] - P_PN[t] * delta_BMm[t]

print(f"{'t':>2} | {'ΔDA':>10} | {'ΔBM+':>12} | {'ΔBM-':>12} || {'Surplus':>10}")
print("-" * 60)
for t in range(T):
    print(f"{t:2} | {delta_DA[t]:10.1f} | {delta_BMp[t]:12.1f} | {delta_BMm[t]:12.1f} || {total_surplus[t]:10.1f}")
print("-" * 60)
print(f"{'Σ':>2} | {delta_DA.sum():>10} | {delta_BMp.sum():>12} | {delta_BMm.sum():>12} || {total_surplus.sum():10.1f}")

 t |        ΔDA |         ΔBM+ |         ΔBM- ||    Surplus
------------------------------------------------------------
 0 |        0.0 |          0.0 |          0.0 ||        0.0
 1 |        0.0 |          0.0 |          0.0 ||        0.0
 2 |        0.0 |          0.0 |          0.0 ||        0.0
 3 |        0.0 |          0.0 |          0.0 ||        0.0
 4 |        0.0 |          0.0 |          0.0 ||        0.0
 5 |        0.0 |          0.0 |          0.0 ||        0.0
 6 |        0.0 |          0.0 |          0.0 ||        0.0
 7 |        1.0 |         -1.0 |          0.0 ||      -16.2
 8 |        8.0 |        -10.0 |         -2.0 ||      286.4
 9 |        5.0 |         -5.0 |          0.0 ||       35.8
10 |       51.0 |        -60.0 |         -9.0 ||     -409.6
11 |        0.0 |          0.0 |          0.0 ||        0.0
12 |        0.0 |          0.0 |          0.0 ||        0.0
13 |        0.0 |          0.0 |          0.0 ||        0.0
14 |       81.0 |        -81.0 |       

In [29]:
delta_DAp = np.where(delta_DA > 0,  delta_DA, 0)
delta_DAm = np.where(delta_DA < 0, -delta_DA, 0)
# print(f"{'t':>2} | {'α':>6} | {'∑x':>6} | {'ΔDA':>8} | {'ΔDA+':>8} | {'ΔDA-':>8}")
# print("-" * 55)
# for t in range(T):
#     print(f"{t:2} | {av[t]:6.2f} | {np.sum(xv[:,t]):6.2f} | {delta_DA[t]:8.2f} | {delta_DAp[t]:8.2f} | {delta_DAm[t]:8.2f}")

E_INp = np.zeros(T)
E_INm = np.zeros(T)

for t in range(T):
    E_INp[t] = max(np.sum(ypv[:, t]) - (bpv[t] + delta_DAp[t]),0)
    E_INm[t] = max(np.sum(ymv[:, t]) - (bmv[t] + delta_DAm[t]),0)
    
print(f"{'t':>2} | {'∑y⁺':>9} | {'β⁺':>9} | {'ΔDA+':>7} | {'E_IN⁺':>8} || {'∑y⁻':>7} | {'β⁻':>6} | {'ΔDA-':>6} | {'E_IN⁻':>7}")
print("-" * 90)
for t in range(T):
    sum_phi_p = np.sum(ypv[:, t])
    sum_phi_m = np.sum(ymv[:, t])
    print(f"{t:2} | {sum_phi_p:9.2f} | {bpv[t]:9.2f} | {delta_DAp[t]:7.2f} | {E_INp[t]:8.2f} ||"
          f" {sum_phi_m:7.2f} | {bmv[t]:6.2f} | {delta_DAm[t]:6.2f} | {E_INm[t]:7.2f}")

 t |       ∑y⁺ |        β⁺ |    ΔDA+ |    E_IN⁺ ||     ∑y⁻ |     β⁻ |   ΔDA- |   E_IN⁻
------------------------------------------------------------------------------------------
 0 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 1 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 2 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 3 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 4 |      1.00 |      1.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 5 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 6 |      1.00 |      1.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 7 |      2.00 |      1.00 |    1.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 8 |     10.00 |      0.00 |    8.00 |     2.00 ||    8.00 |   6.00 |   0.00 |    2.00
 9 |     22.00 |     17.00 |    5.00 | 

In [30]:
m = 1e-16
e_INp = np.zeros((I, T))
e_INm = np.zeros((I, T))

for t in range(T):
    for i in range(I):
        e_INp[i, t] = (ypv[i, t] / (np.sum(ypv[:, t]) + m)) * E_INp[t]
        e_INm[i, t] = (ymv[i, t] / (np.sum(ymv[:, t]) + m)) * E_INm[t]

print(f"{'i':>2} {'t':>2} | {'y⁺':>8} | {'∑y⁺':>8} | {'e_IN⁺':>10} || {'y⁻':>8} | {'∑y⁻':>8} | {'e_IN⁻':>10}")
print("-" * 80)
for t in range(19, 22):
    for i in range(I):
        sum_phi_p = np.sum(ypv[:, t])
        sum_phi_m = np.sum(ymv[:, t])
        print(f"{i:2} {t:2} | {ypv[i, t]:8.2f} | {sum_phi_p:8.2f} | {e_INp[i, t]:10.2f} ||"
              f" {ymv[i, t]:8.2f} | {sum_phi_m:8.2f} | {e_INm[i, t]:10.2f}")

 i  t |       y⁺ |      ∑y⁺ |      e_IN⁺ ||       y⁻ |      ∑y⁻ |      e_IN⁻
--------------------------------------------------------------------------------
 0 19 |    57.00 |   110.00 |       0.00 ||     0.00 |     0.00 |       0.00
 1 19 |     9.00 |   110.00 |       0.00 ||     0.00 |     0.00 |       0.00
 2 19 |    44.00 |   110.00 |       0.00 ||     0.00 |     0.00 |       0.00
 0 20 |     8.00 |    15.00 |       1.07 ||     0.00 |     2.00 |       0.00
 1 20 |     0.00 |    15.00 |       0.00 ||     2.00 |     2.00 |       2.00
 2 20 |     7.00 |    15.00 |       0.93 ||     0.00 |     2.00 |       0.00
 0 21 |     0.00 |     1.00 |       0.00 ||     1.00 |     1.00 |       0.00
 1 21 |     1.00 |     1.00 |       0.00 ||     0.00 |     1.00 |       0.00
 2 21 |     0.00 |     1.00 |       0.00 ||     0.00 |     1.00 |       0.00


In [31]:
delta_INp = np.zeros(T)
delta_INm = np.zeros(T)
Lambda_IN = np.zeros(T)
rho_IN = np.zeros(T)

for t in range(T):
    # if delta_BMp[t] < 0:
    if delta_BMp[t] < 0 and E_INp[t] > 1e-6:
        delta_INp[t] = min(E_INp[t], -delta_BMp[t])
    else:
        delta_INp[t] = 0
    
    # if delta_BMm[t] < 0:
    if delta_BMm[t] < 0 and E_INm[t] > 1e-6:
        delta_INm[t] = min(E_INm[t], -delta_BMm[t])
    else:
        delta_INm[t] = 0
        
    Lambda_IN[t] = P_RT[t] * delta_INp[t] - P_PN[t] * delta_INm[t]
    rho_IN[t] = Lambda_IN[t] / (E_INp[t] + E_INm[t] + m)
    
print(f"{'t':>2} | {'ΔBM+':>7} | {'E_IN+':>7} | {'ΔIN+':>7} || "
      f"{'ΔBM-':>7} | {'E_IN-':>7} | {'ΔIN-':>7} || "
      f"{'Λ_IN':>8} | {'ρ_IN':>8}")
print("-" * 90)

for t in range(T):
    print(f"{t:2} | {delta_BMp[t]:7.2f} | {E_INp[t]:7.2f} | {delta_INp[t]:7.2f} || "
          f"{delta_BMm[t]:7.2f} | {E_INm[t]:7.2f} | {delta_INm[t]:7.2f} || "
          f"{Lambda_IN[t]:8.2f} | {rho_IN[t]:8.2f}")

 t |    ΔBM+ |   E_IN+ |    ΔIN+ ||    ΔBM- |   E_IN- |    ΔIN- ||     Λ_IN |     ρ_IN
------------------------------------------------------------------------------------------
 0 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 1 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 2 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 3 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 4 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 5 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 6 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 7 |   -1.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 8 |  -10.00 |    2.00 |    2.00 ||   -2.00 |    2.00 |    2.00 ||  -148.00 |   -37.00
 9 |   -5.00 |    0.00 |    0.00 ||    

In [32]:
Lambda_b = np.zeros(T)
rho_b = np.zeros(T)
delta_bp = np.zeros(T)
delta_bm = np.zeros(T)
e_bp = np.zeros((I, T))
e_bm = np.zeros((I, T))

for t in range(T):
    delta_bp[t] = delta_BMp[t] - delta_INp[t]
    delta_bm[t] = delta_BMm[t] - delta_INm[t]
    Lambda_b[t] = (P_RT[t] - P_DA[t]) * delta_bp[t] - (P_PN[t] - P_DA[t]) * delta_bm[t]
    
    e_bp[:, t] = ypv[:, t] - e_INp[:, t]
    e_bm[:, t] = ymv[:, t] - e_INm[:, t]
    
    total_b = np.sum(e_bp[:, t] + e_bm[:, t]+m)

    if total_b < 1e-10:
        rho_b[t] = 0
    # Lambda_b[t]는 유지 (반영되지 않음)
    else:
        rho_b[t] = Lambda_b[t] / (total_b + m)
    
print(f"{'i':>2} {'t':>2} | {'y⁺':>7} | {'e_IN⁺':>7} | {'e_b+':>9} || "
      f"{'y⁻':>7} | {'e_IN⁻':>7} | {'e_b-':>9} || {'Λ_b':>10} | {'ρ_b':>8}")
print("-" * 120)

for t in range(19, 22):
    for i in range(I):
        print(f"{i:2} {t:2} | "
              f"{ypv[i, t]:7.4f} | {e_INp[i, t]:7.4f} | {e_bp[i, t]:9.4f} || "
              f"{ymv[i, t]:7.4f} | {e_INm[i, t]:7.4f} | {e_bm[i, t]:9.4f} || "
              f"{Lambda_b[t]:10.2f} | {rho_b[t]:8.2f}")

 i  t |      y⁺ |   e_IN⁺ |      e_b+ ||      y⁻ |   e_IN⁻ |      e_b- ||        Λ_b |      ρ_b
------------------------------------------------------------------------------------------------------------------------
 0 19 | 57.0000 |  0.0000 |   57.0000 ||  0.0000 |  0.0000 |    0.0000 ||     542.65 |     4.93
 1 19 |  9.0000 |  0.0000 |    9.0000 ||  0.0000 |  0.0000 |    0.0000 ||     542.65 |     4.93
 2 19 | 44.0000 |  0.0000 |   44.0000 ||  0.0000 |  0.0000 |    0.0000 ||     542.65 |     4.93
 0 20 |  8.0000 |  1.0667 |    6.9333 ||  0.0000 |  0.0000 |    0.0000 ||     786.96 |    60.54
 1 20 |  0.0000 |  0.0000 |    0.0000 ||  2.0000 |  2.0000 |    0.0000 ||     786.96 |    60.54
 2 20 |  7.0000 |  0.9333 |    6.0667 ||  0.0000 |  0.0000 |    0.0000 ||     786.96 |    60.54
 0 21 |  0.0000 |  0.0000 |    0.0000 ||  1.0000 |  0.0000 |    1.0000 ||      24.51 |    12.26
 1 21 |  1.0000 |  0.0000 |    1.0000 ||  0.0000 |  0.0000 |    0.0000 ||      24.51 |    12.26
 2 21 |  0.0000

In [33]:
lambda_it = np.zeros((I, T))
lambda_sum = np.zeros(T)

for t in range(T):
    for i in range(I):
        lambda_it[i, t] += rho_IN[t] * (e_INp[i, t] + e_INm[i, t])
        lambda_it[i, t] += rho_b[t] * (e_bp[i, t] + e_bm[i, t])
    lambda_sum[t] = np.sum(lambda_it[:, t])     

print(f"{'i':>2} {'t':>2} | {'λ_it':>10}")
print("-" * 20)
for t, i in product(range(20, 23), range(I)):
    print(f"{i:2} {t:2} | {lambda_it[i, t]:10.4f}")

 i  t |       λ_it
--------------------
 0 20 |   360.1189
 1 20 |  -111.7392
 2 20 |   315.1040
 0 21 |    12.2562
 1 21 |    12.2562
 2 21 |     0.0000
 0 22 |     0.0000
 1 22 |     0.0000
 2 22 |     0.0000


In [34]:
print(f"{'t':>2} | {'Λ_IN':>8} | {'Λ_b':>8} || {'∑λ_it':>10} | {'Difference':>10}")
print("-" * 55)

total_IN = 0
total_b  = 0
total_lambda = 0

for t in range(T):
    total_lhs = Lambda_IN[t] + Lambda_b[t]
    diff = total_lhs - lambda_sum[t]

    total_IN     += Lambda_IN[t]
    total_b      += Lambda_b[t]
    total_lambda += lambda_sum[t]

    print(f"{t:2} | {Lambda_IN[t]:8.2f} | {Lambda_b[t]:8.2f} || "
          f"{lambda_sum[t]:10.2f} | {diff:10.2f}")

print("-" * 55)
print(f"{'Σ':>2} | {total_IN:8.2f} | {total_b:8.2f}|| "
      f"{total_lambda:10.2f} | {total_IN + total_b - total_lambda:10.2f}")
print("-" * 55)
print(f"Confirm: {total_surplus.sum()}")

 t |     Λ_IN |      Λ_b ||      ∑λ_it | Difference
-------------------------------------------------------
 0 |     0.00 |    -0.00 ||       0.00 |       0.00
 1 |     0.00 |    -0.00 ||       0.00 |       0.00
 2 |     0.00 |    -0.00 ||       0.00 |       0.00
 3 |     0.00 |    -0.00 ||       0.00 |       0.00
 4 |     0.00 |     0.00 ||       0.00 |       0.00
 5 |     0.00 |    -0.00 ||       0.00 |       0.00
 6 |     0.00 |    -0.00 ||       0.00 |       0.00
 7 |     0.00 |   -16.23 ||     -16.23 |       0.00
 8 |  -148.00 |   434.42 ||     286.41 |       0.00
 9 |     0.00 |    35.75 ||      35.75 |      -0.00
10 |  -424.02 |    14.40 ||    -409.62 |       0.00
11 |     0.00 |     0.00 ||       0.00 |       0.00
12 |     0.00 |     0.00 ||       0.00 |       0.00
13 |     0.00 |     0.00 ||       0.00 |       0.00
14 |     0.00 |  2277.63 ||    2277.63 |       0.00
15 |     0.00 |     0.00 ||       0.00 |       0.00
16 |     0.00 |   269.81 ||     269.81 |       0.00
17 |    

In [35]:
print(f"{'t':>2} || {'E_IN+':>7} | {'ΔIN⁺':>7} | {'Δb⁺':>7} | {'ΔBM⁺':>7} | {'ΔDA+':>7} || {'E_IN-':>7} | {'ΔIN⁻':>7} | {'Δb⁻':>7} | {'ΔBM⁻':>7} | {'ΔDA-':>7}")
print("-" * 105)
for t in range(T):
    print(f"{t:2} || {E_INp[t]:7.2f} | {delta_INp[t]:7.2f} | {delta_bp[t]:7.2f} | {delta_BMp[t]:7.2f} | {delta_DAp[t]:7.2f} || "
          f"{E_INm[t]:7.2f} | {delta_INm[t]:7.2f} | {delta_bm[t]:7.2f} | {delta_BMm[t]:7.2f} | {delta_DAm[t]:7.2f} ")

 t ||   E_IN+ |    ΔIN⁺ |     Δb⁺ |    ΔBM⁺ |    ΔDA+ ||   E_IN- |    ΔIN⁻ |     Δb⁻ |    ΔBM⁻ |    ΔDA-
---------------------------------------------------------------------------------------------------------
 0 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 1 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 2 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 3 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 4 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 5 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 6 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 7 ||    0.00 |    0.00 |   -1.00 |   -1.00 |  

In [36]:
print(f"{'t':>2} | {'ΔDA+':>7} | {'ΔDA-':>7} | {'ΔBM+':>7} | {'ΔBM-':>7} | {'Δb⁺':>7} | {'Δb⁻':>7}")
print("-" * 65)
for t in range(T):
    print(f"{t:2} | {delta_DAp[t]:7.2f} | {delta_DAm[t]:7.2f} | {delta_BMp[t]:7.2f} | {delta_BMm[t]:7.2f} | "
          f"{delta_bp[t]:7.2f} | {delta_bm[t]:7.2f}")

 t |    ΔDA+ |    ΔDA- |    ΔBM+ |    ΔBM- |     Δb⁺ |     Δb⁻
-----------------------------------------------------------------
 0 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 1 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 2 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 3 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 4 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 5 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 6 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 7 |    1.00 |    0.00 |   -1.00 |    0.00 |   -1.00 |    0.00
 8 |    8.00 |    0.00 |  -10.00 |   -2.00 |  -12.00 |   -4.00
 9 |    5.00 |    0.00 |   -5.00 |    0.00 |   -5.00 |    0.00
10 |   51.00 |    0.00 |  -60.00 |   -9.00 |  -69.00 |  -18.00
11 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
12 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
13 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00